In [2]:
import tensorflow as tf

In [3]:
tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

In [8]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [10]:
input_shape = X_train.shape[1:]

In [11]:
import keras
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [12]:
model.compile(loss=huber_fn, optimizer="nadam")

In [13]:
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
363/363 [==============================] - 1s 993us/step - loss: 0.5744 - val_loss: 0.2720
Epoch 2/10
363/363 [==============================] - 0s 793us/step - loss: 0.2150 - val_loss: 0.2368
Epoch 3/10
363/363 [==============================] - 0s 758us/step - loss: 0.2045 - val_loss: 0.2050
Epoch 4/10
363/363 [==============================] - 0s 709us/step - loss: 0.1985 - val_loss: 0.1958
Epoch 5/10
363/363 [==============================] - 0s 712us/step - loss: 0.1939 - val_loss: 0.1935
Epoch 6/10
363/363 [==============================] - 0s 727us/step - loss: 0.1901 - val_loss: 0.1947
Epoch 7/10
363/363 [==============================] - 0s 719us/step - loss: 0.1869 - val_loss: 0.1753
Epoch 8/10
363/363 [==============================] - 0s 777us/step - loss: 0.1836 - val_loss: 0.1728
Epoch 9/10
363/363 [==============================] - 0s 728us/step - loss: 0.1807 - val_loss: 0.1767
Epoch 10/10
363/363 [==============================] - 0s 711us/step - loss: 0.178

In [14]:
import keras
import numpy as np
class LayerNormalizationn(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
    def build(self, input_shape):
        self.alpha = self.add_weight(
            shape=(input_shape[-1:]), name="alpha", initializer="ones"
        )
        self.beta = self.add_weight(shape=(input_shape), name="beta", initializer="zeros")
    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        std_deviation = tf.sqrt(variance + self.eps)
        return self.alpha * (X - mean)/(std_deviation) + self.beta

X = X_train.astype(np.float32)

print(X_train.shape)

(11610, 8)


In [15]:
custom_layer_norm = LayerNormalizationn()

keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=3.9357733e-08>

In [72]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [84]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

X_train_full = X_train_full.astype(np.float32) / 255.0

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test.astype(np.float32) / 255.0


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
def random_batch(X, y, batch_size):
    idx = np.random.randint(0, len(X), size=batch_size)
    return X[idx], y[idx]
batch_size = 32
n_steps = len(X_train) // batch_size
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

optimizer = keras.optimizers.Nadam(learning_rate=0.01)
n_epochs = 5

In [86]:
from tqdm import tqdm
for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}')

    mean_loss.reset_states()
    for step in range(n_steps):
        X_batch, y_batch = random_batch(X_train, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
            gradients = tape.gradient(loss, model.treinable_variables)
            optimizer.apply_gradients(zip(gradients, model.treinable_variables))
            mean_loss(loss)
            status = OrderedDict()
            status["loss"] = mean_loss.result().numpy()
            for metric in metrics:
                metric(y_batch, y_pred)
                status[metric.name] = metric.result.numpy()
            steps.set_postfix(status)
        y_pred = model(X_valid)
        status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
        status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(
            tf.constant(y_valid, dtype=np.float32), y_pred))
        steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

ModuleNotFoundError: No module named 'tqdm'